In [24]:
### basic libraries
import os
import itertools
from datetime import datetime
from json import dumps
import pandas as pd 
import numpy as np

### carto libraries
import shapefile ### not necessary ?
import geopandas as gp
#from shapely.geometry import Polygon
from shapely.geometry import shape, Point

### for plotting in Jupyter
%matplotlib inline
import matplotlib.pyplot as plt
#import seaborn as sns
plt.rcParams["figure.figsize"] = (10.0, 10.0)
#plt.style.use("bmh")
plt.style.use("ggplot")

from app.scripts.topojson import topojson

from pyproj import Proj, transform ### not necessary ?

crs_lambert93 = 2154
crs_WSG84     = 4326
inCRS  = 'epsg:%s' %(crs_lambert93)
outCRS = 'epsg:%s' %(crs_WSG84)
inProj  = Proj(init=inCRS)  # proj in  : Lambert 93
outProj = Proj(init=outCRS) # proj out : WSG 84

### TO DO : find topojson libraries to reduce output file size


In [25]:
### basic folders addresses and names

cwd = os.getcwd()

data_folder      = "app/static/data"

stats_folder     = "stats"
_web             = "_web" 
stats_web_folder = stats_folder + _web

stats_path     = os.path.join(cwd, data_folder, stats_folder)
stats_web_path = os.path.join(cwd, data_folder, stats_web_folder)


carto_folder     = "carto"
_web             = "_web"
carto_web_folder = carto_folder + _web

carto_path     = os.path.join(cwd, data_folder, carto_folder)
carto_web_path = os.path.join(cwd, data_folder, carto_web_folder)



print "-- cwd :"            , cwd
print "-- stats_web_path : ", stats_web_path
print "-- carto_web_path : ", carto_web_path

-- cwd : /Users/jpy/Dropbox/_FLASK/concours_pesticides
-- stats_web_path :  /Users/jpy/Dropbox/_FLASK/concours_pesticides/app/static/data/stats_web
-- carto_web_path :  /Users/jpy/Dropbox/_FLASK/concours_pesticides/app/static/data/carto_web


In [26]:
### read stations_web.csv from : data/stats_web

_sep_csv         = ";"
csv_encoding_web = "utf-8"

stations_filename = "stations_web.csv"
stations_web_filepath = os.path.join(stats_web_path, stations_filename)
print stations_web_filepath

df_stations_web = pd.read_csv( stations_web_filepath, sep=_sep_csv, encoding=csv_encoding_web )

df_stations_web.head(3)

/Users/jpy/Dropbox/_FLASK/concours_pesticides/app/static/data/stats_web/stations_web.csv


,NUM_DEP,NOM_COM,CD_ME_niv1_surf,CD_ME_v2,CD_STATION,NUM_COM,codagence,ALTITUDE,PROFONDEUR_MAXI_POINT,reseau2009,...,fi_ma_2007,fi_ma_2008,fi_ma_2009,fi_ma_2010,fi_ma_2011,fi_ma_2012,fi_ma_2013,fi_ma_2014,LAT_WSG84,LONG_WSG84
0,NaN,NaN,NaN,NaN,10261X0039/F3,NaN,AEAG,10.0,33,Hors RCS et RCO,...,oui,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.363081,-5.983856
1,NaN,NaN,NaN,CG004,01688X0034/AVAL,NaN,AERM,235.0,NaN,Hors RCS et RCO,...,NaN,NaN,oui,NaN,NaN,NaN,NaN,NaN,-1.363081,-5.983856
2,NaN,NaN,NaN,CG004,01688X0039/F1,NaN,AERM,170.0,80,Hors RCS et RCO,...,oui,NaN,oui,NaN,NaN,NaN,NaN,NaN,-1.363081,-5.983856


In [27]:
### create point geometries

geometry     = [ Point(xy) for xy in zip(df_stations_web.LAT_WSG84, df_stations_web.LONG_WSG84) ]

gdf_stations = gp.GeoDataFrame(df_stations_web, crs=crs_WSG84, geometry=geometry)

gdf_stations.tail(3)

,NUM_DEP,NOM_COM,CD_ME_niv1_surf,CD_ME_v2,CD_STATION,NUM_COM,codagence,ALTITUDE,PROFONDEUR_MAXI_POINT,reseau2009,...,fi_ma_2008,fi_ma_2009,fi_ma_2010,fi_ma_2011,fi_ma_2012,fi_ma_2013,fi_ma_2014,LAT_WSG84,LONG_WSG84,geometry
13036,95,VIENNE-EN-ARTHIES,HG107,NaN,01514X0029/P,95656,AESN,100.0,6.5,Hors RCS et RCO,...,NaN,oui,NaN,NaN,NaN,NaN,NaN,1.721380,49.052672,POINT (1.72137996602 49.0526718809)
13037,95,VILLERS-EN-ARTHIES,HG107,NaN,01514X0005/HY,95676,AESN,100.0,NaN,Hors RCS et RCO,...,NaN,oui,NaN,NaN,NaN,NaN,NaN,1.744907,49.090419,POINT (1.74490703083 49.0904192425)
13038,95,WY-DIT-JOLI-VILLAGE,HG107,NaN,01521X0004/HY,95690,AESN,110.0,NaN,Hors RCS et RCO,...,oui,NaN,NaN,NaN,NaN,NaN,NaN,1.838831,49.098160,POINT (1.83883087182 49.0981601538)


In [28]:
#import fiona
#fiona.crs.from_epsg(4326)

In [29]:
### set projection CRS WSG84 

gdf_stations.crs = {
                    'init' : 'epsg:%s' %(crs_WSG84),
                    'no_defs': True
                   }

print gdf_stations.crs


{'init': 'epsg:4326', 'no_defs': True}


In [30]:
# work on copy
gdf_stations_json = gdf_stations.copy()

### set indexes :

gdf_stations_json.set_index([
                        #"NUM_DEP", 
                        #"NUM_COM", 
                        "NOM_COM",  
                        #"CD_ME_niv1_surf", 
                        #"CD_ME_v2", 
                        #"CD_STATION"
                       ], inplace=True) 
#gdf_stations.set_index("CD_STATION", inplace=True) 
gdf_stations_json.sort_index(inplace=True) 

print "-- gdf_stations_json.shape : ", gdf_stations_json.shape

-- gdf_stations_json.shape :  (13039, 25)


In [31]:
gdf_stations_json.head(7)

,NUM_DEP,CD_ME_niv1_surf,CD_ME_v2,CD_STATION,NUM_COM,codagence,ALTITUDE,PROFONDEUR_MAXI_POINT,reseau2009,reseau2010,...,fi_ma_2008,fi_ma_2009,fi_ma_2010,fi_ma_2011,fi_ma_2012,fi_ma_2013,fi_ma_2014,LAT_WSG84,LONG_WSG84,geometry
NOM_COM,,,,,,,,,,,,,,,,,,,,,
ABAINVILLE,55,HG306,HG306,02662X0028/F,55001,AESN,314.00,60,Hors RCS et RCO,NaN,...,NaN,oui,NaN,NaN,NaN,NaN,NaN,5.504758,48.542085,POINT (5.504757967590001 48.5420845455)
ABBANS-DESSOUS,25,DG306,DG306,05292X0014/P,25001,AERM&C,225.00,7.6,Hors RCS et RCO,NaN,...,oui,NaN,NaN,NaN,NaN,NaN,NaN,5.866131,47.132007,POINT (5.866130982540001 47.1320065756)
ABBEVILLE,80,AG011,NaN,00335X0104/F1,80001,AEAP,15.56,37,Hors RCS et RCO,NaN,...,NaN,oui,NaN,NaN,NaN,NaN,NaN,1.851135,50.115745,POINT (1.85113504841 50.1157453806)
ABBEVILLE,80,AG011,NaN,00335X0105/F2,80001,AEAP,16.88,37,Hors RCS et RCO,NaN,...,oui,NaN,NaN,NaN,NaN,NaN,NaN,1.830504,50.121192,POINT (1.83050433317 50.1211918975)
ABBEVILLE,80,AG011,NaN,00335X0103/F,80001,AEAP,14.54,36.8,Hors RCS et RCO,horsRCSRCODRIRE,...,oui,NaN,NaN,NaN,NaN,NaN,NaN,1.832907,50.132955,POINT (1.83290650546 50.1329550142)
ABBEVILLE-LA-RIVIERE,91,GG092,NaN,02931X0019/P,91001,AESN,90.00,34.7,Hors RCS et RCO,horsRCSRCODRIRE,...,oui,NaN,NaN,NaN,NaN,NaN,NaN,2.232801,48.347829,POINT (2.23280108689 48.3478292595)
ABBEVILLERS,25,DG178,NaN,04755X0007/F1,25004,AERM&C,470.00,90,Hors RCS et RCO,NaN,...,oui,NaN,NaN,NaN,NaN,NaN,NaN,6.916196,47.434150,POINT (6.91619556761 47.43415020899999)


In [32]:
print list(gdf_stations_json.columns)

[u'NUM_DEP', u'CD_ME_niv1_surf', u'CD_ME_v2', u'CD_STATION', u'NUM_COM', u'codagence', u'ALTITUDE', u'PROFONDEUR_MAXI_POINT', u'reseau2009', u'reseau2010', u'reseau2011', u'reseau2012', u'reseau2013', u'reseau2014', u'fi_ma_2007', u'fi_ma_2008', u'fi_ma_2009', u'fi_ma_2010', u'fi_ma_2011', u'fi_ma_2012', u'fi_ma_2013', u'fi_ma_2014', u'LAT_WSG84', u'LONG_WSG84', 'geometry']


In [33]:
### drop useless columns for geographic purposes

cols_to_drop = [
    #u'NUM_COM', 
    #u'codagence', 
    #u'ALTITUDE', 
    #u'PROFONDEUR_MAXI_POINT',
    u'reseau2009', u'reseau2010', u'reseau2011', u'reseau2012', u'reseau2013', u'reseau2014', 
    u'fi_ma_2007', u'fi_ma_2008', u'fi_ma_2009', u'fi_ma_2010', u'fi_ma_2011', u'fi_ma_2012', u'fi_ma_2013', u'fi_ma_2014', 
    #u'CD_STATION_', 
    #u'LAT_WSG84', 
    #u'LONG_WSG84', 
    #'geometry'
    ]
gdf_stations_json.drop(cols_to_drop, axis=1, inplace=True)


In [34]:
gdf_stations_json = gdf_stations_json.dropna( axis=0, how="all") # on empty rows


In [39]:
gdf_stations.shape

(13039, 26)

In [35]:
gdf_stations_json.head()

,NUM_DEP,CD_ME_niv1_surf,CD_ME_v2,CD_STATION,NUM_COM,codagence,ALTITUDE,PROFONDEUR_MAXI_POINT,LAT_WSG84,LONG_WSG84,geometry
NOM_COM,,,,,,,,,,,
ABAINVILLE,55,HG306,HG306,02662X0028/F,55001,AESN,314.00,60,5.504758,48.542085,POINT (5.504757967590001 48.5420845455)
ABBANS-DESSOUS,25,DG306,DG306,05292X0014/P,25001,AERM&C,225.00,7.6,5.866131,47.132007,POINT (5.866130982540001 47.1320065756)
ABBEVILLE,80,AG011,NaN,00335X0104/F1,80001,AEAP,15.56,37,1.851135,50.115745,POINT (1.85113504841 50.1157453806)
ABBEVILLE,80,AG011,NaN,00335X0105/F2,80001,AEAP,16.88,37,1.830504,50.121192,POINT (1.83050433317 50.1211918975)
ABBEVILLE,80,AG011,NaN,00335X0103/F,80001,AEAP,14.54,36.8,1.832907,50.132955,POINT (1.83290650546 50.1329550142)


In [36]:
#gdf_stations = gdf_stations.reset_index()
#gdf_stations.head(3)

In [38]:
### save geojson for stations


### filepath for stations_web.geojson
out_filename = "stations_web_carto.geojson"
out_filepath = os.path.join(carto_web_path, out_filename )


### overule Fiona overwrite bug
try: 
    os.remove(out_filepath)
except OSError:
    pass


#fiona.supported_drivers
gdf_stations_json.to_file( out_filepath, driver="GeoJSON" )

